In [ ]:
!pip install chromadb

In [2]:
import pandas as pd
text_data = pd.read_csv(r"C:\Users\dhwan\Desktop\clg\LLM\CIA\1k_stories_100_genre.csv")
text_data

,id,title,story,genre
0,457580,The Chronicles of the Cosmic Rift,"In the year 2250, Earth had made significant s...",Science Fiction
1,297904,Eldoria's Enchanted Whispers,"In a land far away, where the sun shone bright...",Fantasy
2,620436,Echoes of Whispered Shadows,"Once upon a time, in a small, tranquil town ca...",Mystery
3,634687,Emerald Amulet Chronicles Revealed,"Once upon a time in the 16th century, a small ...",Historical Adventure
4,513427,The Shadows of St. Augustine,In the sun-drenched coastal city of St. August...,Thriller
...,...,...,...,...
995,489018,The Asteroids Secret,"In the distant year of 3025, the spacecraft Se...",Space Exploration
996,689389,The Gears of Destiny,"In the steam-powered city of Icarus, the clock...",Steampunk Fantasy
997,157117,The Great Heist,"Jack, a small-time crook, hatched a plan to st...",Noir Comedy
998,170613,The Unseen World,"In a small village, a curious phenomenon occur...",Social Commentary


In [3]:
story_seq = ['genre', 'title', 'full_text']

def preprocess_text(text):
    # Remove newlines and extra spaces
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = ' '.join(text.split())
    return text

text_data['full_text'] = text_data['story'].apply(preprocess_text)

In [4]:
text_data.head()

,id,title,story,genre,full_text
0,457580,The Chronicles of the Cosmic Rift,"In the year 2250, Earth had made significant s...",Science Fiction,"In the year 2250, Earth had made significant s..."
1,297904,Eldoria's Enchanted Whispers,"In a land far away, where the sun shone bright...",Fantasy,"In a land far away, where the sun shone bright..."
2,620436,Echoes of Whispered Shadows,"Once upon a time, in a small, tranquil town ca...",Mystery,"Once upon a time, in a small, tranquil town ca..."
3,634687,Emerald Amulet Chronicles Revealed,"Once upon a time in the 16th century, a small ...",Historical Adventure,"Once upon a time in the 16th century, a small ..."
4,513427,The Shadows of St. Augustine,In the sun-drenched coastal city of St. August...,Thriller,In the sun-drenched coastal city of St. August...


In [5]:
text_data = text_data[story_seq]
text_data 

,genre,title,full_text
0,Science Fiction,The Chronicles of the Cosmic Rift,"In the year 2250, Earth had made significant s..."
1,Fantasy,Eldoria's Enchanted Whispers,"In a land far away, where the sun shone bright..."
2,Mystery,Echoes of Whispered Shadows,"Once upon a time, in a small, tranquil town ca..."
3,Historical Adventure,Emerald Amulet Chronicles Revealed,"Once upon a time in the 16th century, a small ..."
4,Thriller,The Shadows of St. Augustine,In the sun-drenched coastal city of St. August...
...,...,...,...
995,Space Exploration,The Asteroids Secret,"In the distant year of 3025, the spacecraft Se..."
996,Steampunk Fantasy,The Gears of Destiny,"In the steam-powered city of Icarus, the clock..."
997,Noir Comedy,The Great Heist,"Jack, a small-time crook, hatched a plan to st..."
998,Social Commentary,The Unseen World,"In a small village, a curious phenomenon occur..."


In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# Initialize ChromaDB client and collection
client = chromadb.PersistentClient(r"C:\Users\dhwan\Desktop\clg\LLM\CIA")
collection = client.create_collection("1k_stories_100_genre.csv")
# Prepare documents for ingestion
documents = text_data['full_text'].tolist()
metadatas = text_data[['genre', 'title']].to_dict(orient='records')
ids = [str(i) for i in text_data.index]

# Use default embedding function (or specify one if available)
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [8]:
# Example: Retrieve relevant stories for a query
query = "A story about a brave knight"
results = collection.query(
    query_texts=[query],
    n_results=100
)

# Display retrieved stories' titles and genres
for res in results['metadatas'][0]:
    print(f"Title: {res['title']}, Genre: {res['genre']}")
    result = results['documents'][0]
    # print(f"Text: {result['full_text'][:200]}...")  # Display first 200 characters of the story



In [9]:
import requests
def list_ollama_models():
    url = "http://localhost:11434/api/tags"
    response = requests.get(url)
    response.raise_for_status()
    return [m['name'] for m in response.json().get('models', [])]

print(list_ollama_models())


['mistral:latest', 'llama2:latest']


In [10]:
def rag_prompt(topic, length=500):
    """
    Generate a prompt for a text generation task.

    Args:
        topic (str): The topic of the prompt.
        length (int, optional): The desired length of the prompt. Defaults to 100.

    Returns:
        str: The generated prompt.
    """
    prompt = f"Write a {length}-word story on the topic of {topic}. "
    # prompt += "Please provide a clear and concise argument, supported by relevant examples and evidence."
    return prompt

rag_prompt_text = rag_prompt("A brave knight", length=1000)

In [12]:
import requests

def query_ollama(prompt, model="gemma:2b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()["response"]
    except requests.exceptions.HTTPError as e:
        print(f"HTTPError: {e}")
        print("Is the Ollama server running and is the model available?")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

if 'rag_prompt' not in globals():
    raise NameError("rag_prompt is not defined. Please run the cell that defines 'rag_prompt' first.")

ollama_response = query_ollama(rag_prompt_text)
print(ollama_response)

HTTPError: 404 Client Error: Not Found for url: http://localhost:11434/api/generate
Is the Ollama server running and is the model available?
None
